In [2]:
#Install required packages if needed
#!conda install -c conda-forge lxml --yes
#!conda install -c anaconda pandasql --yes
print('Package Installation is complete')

Package Installation is complete


The following web scraping code is based on tutorial: "Web Scraping HTML Tables with Python" by Syed Sadat Nazrul
https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059  I left most of Syed's comments in the code intact.

xpath of the table in the wikipedia document was obtained using "COPY"->"COPY XPath" in the Elements tab of the Google Chrome "Developer Tools" utility
lxml.html library home page: https://lxml.de/lxmlhtml.html

In [141]:
import requests
import lxml.html as lh
import pandas as pd
##############################
# Scrape the data
##############################
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr')

##############################
# Organize the data into lists
##############################
#tr_elements is a list of HtmlElement (Element tr) objects 
#get the shape of the table (num_rows,num_columns)
num_rows=len(tr_elements)
num_columns=len(tr_elements[0])

col=[]

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    name=t.text_content().strip()
    col.append((name,[]))

#Since the first row is the header, data is stored on the second row onwards
for j in range(1,num_rows):
    #T is our j'th row
    T=tr_elements[j]

    column_index=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().strip()
        #Append the data to the empty list of each column
        col[column_index][1].append(data)
        #Increment column_index for the next column
        column_index+=1

##############################
# Create the Pandas dataframe
##############################

Dict={title:column for (title,column) in col}

df=pd.DataFrame(Dict)
df.head(4)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


Clean up the dataframe
    -remove rows Borough values of 'Not assigned'
    -replace Neghbourhood values of 'Not assigned' with the Borough value

In [107]:
#drop dataframe row if Borough column value is 'Not assigned'
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)

#replace Neighbourhood value with Borough value if Neighbourhood value is 'Not assigned'
indices=df[df['Neighbourhood']=='Not assigned'].index
df.loc[indices, 'Neighbourhood']=df.loc[indices, 'Borough']

Group Neighbourhoods with the same Postcodes

In [140]:
df1=df.groupby(['Postcode','Borough']).Neighbourhood.apply(lambda x: ', '.join(x)).reset_index()

#override the default Pandas dataframe print option that truncates the output table rows and column values
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth',150)
df1


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Print out the shape of the resulting DataFrame (df1)

In [143]:
df1.shape

(103, 3)

Load and prep the csv data containing the Postal codes and Latitude-Longitude coordinates
I originall used a technique for streaming very large files documented on the following stackoverflow URLs:
    -https://stackoverflow.com/questions/35371043/use-python-requests-to-download-csv
    -https://stackoverflow.com/questions/18897029/read-csv-file-from-url-into-python-3-x-csv-error-iterator-should-return-str/18897408
     
...Then found the pd.read_csv(URL) function could create a dataframe directly from the URL string.
I renamed column label: 'Postal Code' to Postcode to simpify the join query in the next cell.

In [133]:
import csv
import requests

CSV_URL = 'https://cocl.us/Geospatial_data'
df_ll = pd.read_csv(CSV_URL)
df_ll = df_ll.rename(columns={'Postal Code': 'Postcode'})
df_ll.head(4)


,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917


Create a new dataframe using the pandasql.sqdf function to join the the first Postcode-Borough-Neighborhood dataframe (df1) with the Postcode-Latitude-Longitude dataframe (df_ll)

In [132]:
from pandasql import sqldf
sql="select a.*, b.Latitude, b.Longitude from df1 a INNER JOIN df_ll b ON a.Postcode=b.Postcode;"
df3=sqldf(sql)

df3

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Print out the shape of the merged DataFrame (df3)

In [145]:
df3.shape

(103, 5)

Visualization of Toronto postal code data

In [159]:
Etobicoke_df=sqldf("select * from df3 where Borough='Etobicoke';")
Etobicoke_df.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",43.636258,-79.498509
4,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor",43.628841,-79.520999


Create Plot of Toronto Postcode zones
    mean geoposition of all Toronto Postcodes (large red circle)
    all Postcode centers (blue circles)
    Postcode locations in the Etobicoke Borough (blue circles with smaller orange center)

In [168]:
import folium # plotting library
latitude=df3["Latitude"].mean()
longitude=df3["Longitude"].mean()
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around Ecco


# add Toronto center as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Toronto mean postalcode geoposition',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map)

# add the toronto postcode locations
for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Postcode']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        poup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# add Etobicoke postalcode points
for lat, lng, label in zip(Etobicoke_df['Latitude'], Etobicoke_df['Longitude'], Etobicoke_df['Postcode']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=3,
        poup=label,
        fill=True,
        color='orange',
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(venues_map)





In [170]:
# display map
venues_map